In [1]:
#time in traffic value is in seconds text is in minutes

In [79]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from census import Census
import gmaps
import requests
import json
from pprint import pprint
from config import (api_key, gkey)
c = Census(api_key, year=2016)
from datetime import datetime

In [80]:
file_to_load = pd.read_csv("bayarea_zipcodes.csv", dtype="object", encoding="utf-8")
zip_code_df = pd.DataFrame(file_to_load)
zip_code_df.head()
bay_area_zip = []
for index, row in zip_code_df.iterrows():
    bay_area_zip.append(row['ZIP'])


In [81]:
time_list = []
time_am= int(datetime(2018,10,8,7).timestamp())
time_pm = int(datetime(2018,10,8,16).timestamp())
for x in range(1,7) :
    time= datetime(2018,10,8,7, 0).timestamp()
    time_list.append(time_am+1800)
    time_list.append(time_pm+1800)
    #print(time)
time_list.append(time)
time_list.append(time_pm)
print(time_list)

#datetime.datetime(time_list[0]).timestamp()

[1539009000, 1539041400, 1539009000, 1539041400, 1539009000, 1539041400, 1539009000, 1539041400, 1539009000, 1539041400, 1539009000, 1539041400, 1539007200.0, 1539039600]


In [83]:
#testing variable so im not running long code
#start_loc = ["Alameda,CA","Solano","Sonoma","Napa County"]
#start_loc = ["Alameda,CA","Contra Costa,CA","Marin","Napa County","San Francisco","San Mateo","Santa Clara","Solano","Sonoma"]
#end_loc = (list(reversed(start_loc)))
end_loc = (list(reversed(bay_area_zip)))
origin_Loc = []
destination_loc = []
trip_distance = []
trip_duration = []
traffic_duration = []
#target_url = (f'https://maps.googleapis.com/maps/api/distancematrix/json?origins=San Francisco,CA|Oakland,CA&destinations=San Jose,CA|Palo Alto,CA&departure_time=now&key={gkey}')

In [85]:
for time in time_list:

    for s_loc in bay_area_zip:
        for e_loc in end_loc:
            try:
                target_url = (f'https://maps.googleapis.com/maps/api/distancematrix/json?origins={s_loc}&destinations={e_loc},traffic_model=pessimistic,CA&departure_time={time}&key={gkey}')
                #target_url2 = (f'https://maps.googleapis.com/maps/api/distancematrix/json?origins={e_loc}&destinations={s_loc},CA&departure_time={time}&key={gkey}')
                geo_data = requests.get(target_url).json()
                #geo_data2 = requests.get(target_url2).json()
                destination_loc.append(geo_data['destination_addresses'][0])
                origin_Loc.append(geo_data['origin_addresses'][0])
                trip_distance.append(geo_data['rows'][0]['elements'][0]['distance']['value'])
                traffic_duration.append(geo_data['rows'][0]['elements'][0]['duration_in_traffic']['value'])
                trip_duration.append(geo_data['rows'][0]['elements'][0]['duration']['value'])
                pprint(geo_data)
               
            except(KeyError):
                print(f"something broke with either {s_loc} or {e_loc}")

{'destination_addresses': ['Sunnyvale, CA 94087, USA'],
 'origin_addresses': ['Spanish Flat, CA 94558, USA'],
 'rows': [{'elements': [{'distance': {'text': '158 km', 'value': 157775},
                         'duration': {'text': '1 hour 40 mins', 'value': 6003},
                         'duration_in_traffic': {'text': '2 hours 10 mins',
                                                 'value': 7813},
                         'status': 'OK'}]}],
 'status': 'OK'}
{'destination_addresses': ['Los Altos, CA 94024, USA'],
 'origin_addresses': ['Spanish Flat, CA 94558, USA'],
 'rows': [{'elements': [{'distance': {'text': '145 km', 'value': 144948},
                         'duration': {'text': '1 hour 40 mins', 'value': 6005},
                         'duration_in_traffic': {'text': '2 hours 10 mins',
                                                 'value': 7782},
                         'status': 'OK'}]}],
 'status': 'OK'}
{'destination_addresses': ['Sunnyvale, CA 94086, USA'],
 'origin_a

KeyboardInterrupt: 

In [86]:
#converting units into minutes/miles
#minute_traffic = (traffic_duration-trip_duration)/60
traffic_duration = [x/60 for x in traffic_duration]
trip_duration  = [x/60 for x in trip_duration]
minute_traffic = [x-y for  x,y in zip(traffic_duration,trip_duration)]
#miles_distance = trip_distance/1,609.344
miles_distance = [z* 0.00062137 for z in trip_distance]
#print(miles_distance)

In [87]:
traffic_dic = {"Origin":origin_Loc,
               "Destination":destination_loc,
               "Distance(Miles)":miles_distance,
               "Trip length(minutes)":trip_duration,
               "Trip in traffic(minutes)":traffic_duration,
               "Traffic(minutes)":minute_traffic
              }
traffic_df = pd.DataFrame(traffic_dic).round(2)
traffic_df

,Origin,Destination,Distance(Miles),Trip length(minutes),Trip in traffic(minutes),Traffic(minutes)
0,"Spanish Flat, CA 94558, USA","Sunnyvale, CA 94087, USA",98.04,100.05,130.22,30.17
1,"Spanish Flat, CA 94558, USA","Los Altos, CA 94024, USA",90.07,100.08,129.70,29.62
2,"Spanish Flat, CA 94558, USA","Sunnyvale, CA 94086, USA",88.26,95.65,135.23,39.58
3,"Spanish Flat, CA 94558, USA","Sunnyvale, CA 94085, USA",87.19,92.42,131.82,39.40
4,"Spanish Flat, CA 94558, USA","Los Altos, CA 94022, USA",89.91,98.02,127.77,29.75
5,"Spanish Flat, CA 94558, USA","Mountain View, CA 94040, USA",89.36,94.95,134.90,39.95
6,"Spanish Flat, CA 94558, USA","Menlo Park, CA 94028, USA",85.08,96.48,126.18,29.70
7,"Spanish Flat, CA 94558, USA","Palo Alto, CA 94306, USA",80.66,98.72,132.82,34.10
8,"Spanish Flat, CA 94558, USA","MOFFETT FIELD, CA 94035, USA",87.65,93.88,134.55,40.67
9,"Spanish Flat, CA 94558, USA","Palo Alto, CA 94305, USA",85.41,97.18,128.78,31.60


In [45]:
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E","B25077_001E"),{'for': 'zip code tabulation area:*'})
#state_zipcode
#{'for': 'zip code tabulation area:*'}
census_pd = pd.DataFrame(census_data)
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count","NAME": "Name", "zip code tabulation area": "ZIP",
                                      "B25077_001E":"Median home value"})


census_pd.head()

,Median Age,Population,Poverty Count,Household Income,Per Capita Income,Unemployment Count,Median home value,Name,ZIP
0,45.0,17423.0,1462.0,56714.0,30430.0,479.0,202800.0,ZCTA5 01001,01001
1,23.2,29970.0,8351.0,48923.0,26072.0,1271.0,344000.0,ZCTA5 01002,01002
2,19.9,11296.0,54.0,2499.0,3829.0,1021.0,-666666666.0,ZCTA5 01003,01003
3,44.1,5228.0,230.0,70568.0,32169.0,153.0,213700.0,ZCTA5 01005,01005
4,42.5,14888.0,1410.0,80502.0,36359.0,543.0,258000.0,ZCTA5 01007,01007


In [76]:
city_census =pd.merge(census_pd,zip_code_df,how= 'right',on='ZIP')
new_df =city_census.drop(columns=["Poverty Count","Name","Median Age","the_geom","Unemployment Count"],axis=1)
new_df.head()

,Population,Household Income,Per Capita Income,Median home value,ZIP,PO_NAME,STATE,Area__,Length__
0,27116.0,119530.0,61303.0,1070100.0,94002,BELMONT,CA,163210029.225,86346.4872109
1,4568.0,91127.0,49198.0,699800.0,94005,BRISBANE,CA,132357364.26,67781.5039169
2,42569.0,125768.0,79594.0,1795900.0,94010,BURLINGAME,CA,359016202.637,141377.757027
3,48387.0,75658.0,27786.0,585800.0,94014,DALY CITY,CA,181555330.136,81604.6288525
4,64712.0,81956.0,32345.0,629600.0,94015,DALY CITY,CA,159483452.372,75079.2856064
